In [106]:
import logging
logging.getLogger("paramiko").setLevel(logging.WARNING)

In [35]:
from jnpr.junos import Device

In [36]:
from lxml import etree

In [40]:
#dev = Device(host='xxxx', user='demo', password='demo123', gather_facts=False)
dev = Device(host='xxxx', user='demo', password='demo123')

In [ ]:
dev.open()

In [ ]:
dev.facts

In [ ]:
op = dev.rpc.get_interface_information()
#op = dev.rpc.get_interface_information({'format': 'text'})
#op = dev.rpc.get_interface_information(interface_name='lo0', terse=True)

In [ ]:
print etree.tostring(op)

In [ ]:
print dev.cli("show version", warning=False)

In [ ]:
cnf = dev.rpc.get_config()

In [ ]:
print etree.tostring(cnf)

In [ ]:
data = dev.rpc.get_config(filter_xml=etree.XML('<configuration><system><services/></system></configuration>'))

In [ ]:
print etree.tostring(data)

In [ ]:
from jnpr.junos.utils.fs import FS

In [ ]:
fs = FS(dev)

In [ ]:
fs.ls('/var/tmp')

In [ ]:
print fs.cat('/var/tmp/nitin.log')

### Playing with table/View
```
RouteTable:
  rpc: get-route-information
  args_key: destination
  item: route-table/rt 
  key: rt-destination
  view: RouteTableView

RouteTableView:
  groups:
    entry: rt-entry
  fields_entry:
    # fields taken from the group 'entry'
    protocol: protocol-name
    via: nh/via | nh/nh-local-interface
    age: { age/@seconds : int }
    nexthop: nh/to
```

In [ ]:
from jnpr.junos.op.routes import RouteTable
tbl = RouteTable(dev)
tbl.get()

In [ ]:
for item in tbl:
    print 'protocol:', item.protocol
    print 'age:', item.age
    print 'via:', item.via
    print

In [ ]:
from jnpr.junos.factory.factory_loader import FactoryLoader
import yaml

yaml_data="""
---
ArpTable:
  rpc: get-arp-table-information
  item: arp-table-entry
  key: mac-address
  view: ArpView

ArpView:
  fields:
    mac_address: mac-address
    ip_address: ip-address
    interface_name: interface-name
"""
globals().update(FactoryLoader().load(yaml.load(yaml_data)))
arps = ArpTable(dev)
arps.get()
for arp in arps:
        print 'mac_address: ', arp.mac_address
        print 'ip_address: ', arp.ip_address
        print 'interface_name:', arp.interface_name
        print

## Loading Configs

In [11]:
from jnpr.junos.utils.config import Config
cu = Config(dev)

In [ ]:
data = """interfaces { 
    ge-1/0/1 {
        description "MPLS interface";
        unit 0 {
            family mpls;
        }      
    } 
    ge-1/0/2 {
        description "MPLS interface";
        unit 0 {
            family mpls;
        }      
    }   
}
protocols {
    mpls { 
        interface ge-1/0/1; 
        interface ge-1/0/2;            
    }
}
"""

In [ ]:
cu.load(data, format='text')
cu.commit_check()

In [ ]:
cu.load?

In [ ]:
data = """<policy-options>
          <policy-statement action="delete">
            <name>F5-in</name>
            <term>
                <name>test</name>
                <then>
                    <accept/>
                </then>
            </term>
            <from>
                <protocol>mpls</protocol>
            </from>
        </policy-statement>
        </policy-options>"""


cu.load(data)
cu.commit_check()

In [5]:
xml_temp="""<policy-options>
        <policy-statement>
            <name>all-local</name>
            {% for prot in protocols %}
            <term>
                <name>{{ prot['name'] }}</name>
                <from>
                    <protocol>{{ prot.protocol }}</protocol>
                </from>
                <then>
                    <accept/>
                </then>
            </term>{% endfor %}
        </policy-statement>
    </policy-options>"""


In [6]:
from jinja2 import Template
tmpl = Template(xml_temp)
conf = tmpl.render(protocols=[{'name':'1', 'protocol':'direct'}, {'name':'2', 'protocol':'static'}])
print conf
#cu.load(str(conf))

<policy-options>
        <policy-statement>
            <name>all-local</name>
            
            <term>
                <name>1</name>
                <from>
                    <protocol>direct</protocol>
                </from>
                <then>
                    <accept/>
                </then>
            </term>
            <term>
                <name>2</name>
                <from>
                    <protocol>static</protocol>
                </from>
                <then>
                    <accept/>
                </then>
            </term>
        </policy-statement>
    </policy-options>


In [31]:
%cat /Users/nitinkr/Coding/pyezex/protocol.conf

interfaces { 
    {% for item in interfaces %}
    {{ item }} {
        description "{{ description }}";
        unit 0 {
            family {{ family }};
        }      
    } {% endfor %}    
}
protocols {
    mpls { 
        {% for item in interfaces %} 
        interface {{ item }}; 
        {% endfor %}    
    }
}



In [32]:
%cat /Users/nitinkr/Coding/pyezex/protocol_data.yml

---
interfaces:
  - ge-1/0/1
  - ge-1/0/2
description: 'MPLS interface'
family: mpls


In [33]:
import yaml
data = yaml.load(open('/Users/nitinkr/Coding/pyezex/protocol_data.yml'))
print data

{'interfaces': ['ge-1/0/1', 'ge-1/0/2'], 'description': 'MPLS interface', 'family': 'mpls'}


In [34]:
from jinja2 import Template
tmpl = Template(open('/Users/nitinkr/Coding/pyezex/protocol.conf').read())
conf = tmpl.render(data)
print conf

interfaces { 
    
    ge-1/0/1 {
        description "MPLS interface";
        unit 0 {
            family mpls;
        }      
    } 
    ge-1/0/2 {
        description "MPLS interface";
        unit 0 {
            family mpls;
        }      
    }     
}
protocols {
    mpls { 
         
        interface ge-1/0/1; 
         
        interface ge-1/0/2; 
            
    }
}



In [13]:
cu.load(template_path='/Users/nitinkr/Coding/pyezex/protocol.conf',
        template_vars=data, format='text')

<Element load-configuration-results at 0x1055babd8>

In [14]:
cu.pdiff()


[edit interfaces]
+   ge-1/0/1 {
+       description "MPLS interface";
+       unit 0 {
+           family mpls;
+       }
+   }
+   ge-1/0/2 {
+       description "MPLS interface";
+       unit 0 {
+           family mpls;
+       }
+   }
[edit]
+  protocols {
+      mpls {
+          interface ge-1/0/1.0;
+          interface ge-1/0/2.0;
+      }
+  }



In [15]:
cu.rollback()

True

In [16]:
cu.pdiff()

None


## Software

```python
def update_progress(dev, report):
    print dev.hostname, '> ', report
    
sw.install('/var/tmp/junos-srxsme-12.1X46-D15.3-domestic.tgz', validate=True, progress=update_progress)
```

In [19]:
from jnpr.junos.utils.sw import SW
sw = SW(dev)

In [20]:
sw.install?

In [ ]:
dev = Device(host='xxxx', user='demo', password='demo123', gather_facts=False)
dev.open()
sw = SW(dev)
ok = sw.install(package=r'/Users/nitinkr/Downloads/jinstall-xxxxx-domestic.tgz', progress=update_progress)
if ok:
    print 'rebooting'
    sw.reboot()

In [21]:
from jnpr.junos.utils.scp import SCP

In [23]:
%cat info.txt

Hello world


In [24]:
%rm info.txt

In [ ]:
import time
from bokeh.plotting import figure, output_server, cursession, show
from bokeh.models import NumeralTickFormatter

from jnpr.junos import Device

# prepare output to server
output_server("animated_line")

p = figure(plot_width=600, plot_height=600)
dev = Device(host='xxxx', user='demo', password='demo123', gather_facts=False, port=22)
dev.open()

x_tmp = [0]*5
x_var = [0]*5
ct = time.localtime()
ct = ct.tm_hour*3600+ct.tm_min*60+ct.tm_sec
op = dev.rpc.get_statistics_information(tcp=True)
packets_sent_new = op.xpath('.//packets-sent')[0].text.strip()
packets_recv_new = op.xpath('.//packets-received')[0].text.strip()
p.line([ct, ct+2, ct+4, ct+6, ct+8], x_tmp, name='ex_line',  legend = 'packets-sent')
p.line([ct, ct+2, ct+4, ct+6, ct+8], x_var, name='ex_line', line_color="red", legend = 'packets-recv')
p.xaxis[0].formatter = NumeralTickFormatter(format='00:00:00')
show(p)

# create some simple animation..
# first get our figure example data source
renderer = p.select(dict(name="ex_line"))
ds1 = renderer[0].data_source
ds2 = renderer[1].data_source
while True:
    op = dev.rpc.get_statistics_information(tcp=True)
    packets_sent_new, packets_sent_old = op.xpath('.//packets-sent')[0].text.strip(), packets_sent_new
    packets_recv_new, packets_recv_old = op.xpath('.//packets-received')[0].text.strip(), packets_recv_new
    ct = time.localtime()
    ct = ct.tm_hour*3600+ct.tm_min*60+ct.tm_sec
    ds2.data["x"] = ds1.data["x"] = [ct, ct+2, ct+4, ct+6, ct+8]
    ds1.data["y"] = ds1.data["y"][1:]+[int(packets_sent_new)-int(packets_sent_old)]
    ds2.data["y"] = ds2.data["y"][1:]+[int(packets_recv_new)-int(packets_recv_old)]
    cursession().store_objects(ds1, ds2)
    time.sleep(1.5)